In [63]:
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import tqdm
import ast

In [64]:
df = pd.read_csv("data/cleared_train.csv")

In [65]:
df_pca = df.groupby(['Sensor']).mean().drop(columns=['Day', "Unnamed: 0"],axis=1).transpose()
df_pca = df_pca.rename(columns={"Sensor":"Time"})
df_pca

Sensor,Sensor_1,Sensor_10,Sensor_100,Sensor_101,Sensor_102,Sensor_103,Sensor_104,Sensor_105,Sensor_106,Sensor_107,...,Sensor_958,Sensor_959,Sensor_96,Sensor_960,Sensor_961,Sensor_962,Sensor_963,Sensor_97,Sensor_98,Sensor_99
Time_1,0.018636,0.023155,0.008328,0.007599,0.016455,0.012060,0.018191,0.006955,0.019456,0.015433,...,0.019255,0.025540,0.009373,0.021288,0.021727,0.020037,0.022980,0.016089,0.019061,0.011948
Time_2,0.018240,0.022252,0.007456,0.006854,0.015287,0.011548,0.017057,0.006408,0.018508,0.014245,...,0.017930,0.020264,0.008557,0.019082,0.020601,0.018478,0.021678,0.015761,0.017788,0.011846
Time_3,0.019257,0.021544,0.006888,0.006592,0.014439,0.010566,0.016425,0.006028,0.018064,0.012330,...,0.019030,0.020862,0.008085,0.018992,0.020369,0.017861,0.020859,0.014052,0.017645,0.011569
Time_4,0.017563,0.019948,0.006861,0.005825,0.011932,0.009591,0.015003,0.004847,0.016235,0.012390,...,0.019746,0.022680,0.007106,0.017330,0.018502,0.016348,0.019469,0.012321,0.015824,0.010630
Time_5,0.017388,0.020161,0.006016,0.005849,0.011901,0.009234,0.014779,0.004622,0.015932,0.012503,...,0.019042,0.021634,0.006949,0.016366,0.018376,0.015333,0.019380,0.011885,0.015573,0.010257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Time_140,0.024236,0.031430,0.012930,0.012275,0.027235,0.019606,0.025756,0.012018,0.027051,0.021518,...,0.023093,0.028715,0.015629,0.029527,0.030181,0.029027,0.031135,0.024695,0.026923,0.019426
Time_141,0.022207,0.029907,0.012477,0.011401,0.022973,0.018112,0.024153,0.011016,0.024936,0.016033,...,0.024106,0.027830,0.014321,0.027622,0.027224,0.026757,0.029247,0.022567,0.025474,0.016601
Time_142,0.021496,0.027632,0.010717,0.010384,0.020835,0.016388,0.021949,0.009717,0.023294,0.015181,...,0.020819,0.028226,0.013164,0.024634,0.024559,0.023869,0.026294,0.020313,0.023516,0.015303
Time_143,0.020680,0.027330,0.010437,0.010237,0.019988,0.015634,0.021743,0.009394,0.023016,0.016027,...,0.020105,0.027234,0.012529,0.024761,0.024239,0.023975,0.025772,0.021085,0.023004,0.015423


In [66]:
X_train = df.drop(columns=["Unnamed: 0", "Day"])
y_train = df["Day"]

fets_train = X_train.drop("Sensor", axis=1).columns
sensors_train = X_train.Sensor
X_train = df.loc[:, fets_train].values

X_train = StandardScaler().fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = fets_train)

In [67]:
X_train

,Time_1,Time_2,Time_3,Time_4,Time_5,Time_6,Time_7,Time_8,Time_9,Time_10,...,Time_135,Time_136,Time_137,Time_138,Time_139,Time_140,Time_141,Time_142,Time_143,Time_144
0,-0.232196,-0.621891,-0.490520,-0.330289,-0.364737,-0.229263,-0.239519,-0.355352,-0.175172,-0.238840,...,-0.858966,-0.679798,-0.631901,-0.693973,-0.651974,-0.584317,-0.499363,-0.583953,-0.573618,-0.425320
1,-0.526408,-0.833342,-0.510494,-0.583125,-0.487636,-0.335308,-0.368685,-0.489337,-0.376868,-0.461395,...,-0.753698,-0.806950,-0.869588,-0.836317,-0.575223,-0.632444,-0.751354,-0.353928,-0.633552,-0.667334
2,-0.637554,-0.602067,-0.457228,-0.432790,-0.419359,-0.469632,-0.318454,-0.313041,-0.293408,-0.266659,...,-0.490526,-0.719259,-0.654324,-0.496530,-0.580020,-0.695010,-0.677239,-0.563951,-0.518678,-0.471663
3,0.349688,0.514662,0.548169,0.400887,0.263412,0.548399,0.463721,0.589592,0.680295,0.929576,...,-0.275603,-0.171189,-0.053380,0.086618,0.005206,0.195347,-0.020086,-0.088899,0.025728,-0.075172
4,0.683128,0.765760,0.914374,0.728891,0.850595,0.689792,0.944507,0.956287,0.986316,0.894802,...,0.101609,0.219037,0.130492,0.173861,0.005206,0.103905,0.068852,0.211135,0.145597,0.228633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257116,-0.258348,-0.304713,-0.317405,-0.316622,-0.303288,-0.314099,-0.332805,-0.348300,-0.328183,-0.336208,...,-0.218582,-0.228188,-0.196889,-0.280719,-0.292204,-0.266676,-0.281959,-0.253917,-0.214011,-0.209052
257117,0.970801,1.182056,1.020906,1.009062,1.389984,1.333130,1.009090,1.287723,1.320157,1.339913,...,0.693745,0.701339,0.569988,0.903943,0.609621,0.421546,0.597539,0.761195,0.690002,0.784750
257118,-0.519869,-0.066830,0.055458,-0.309789,-0.118940,-0.349448,0.226916,-0.277782,0.026523,-0.009330,...,-0.433506,-0.508800,-0.282097,-0.551630,-0.196265,-0.415871,-0.173257,-0.178908,-0.428776,-0.147261
257119,0.153547,-0.251850,0.055458,-0.186787,-0.091629,-0.130288,-0.002713,-0.157901,-0.043027,0.004580,...,0.360395,0.447034,0.260547,0.137127,0.312210,0.103905,-0.029968,0.296144,0.255477,0.012365


In [68]:
from sklearn.decomposition import PCA

In [69]:
pca = PCA(n_components="mle", svd_solver = 'full')

pcas_train = pca.fit_transform(X_train)
pca_df_train = pd.DataFrame(data=pcas_train)
pca_df_train

,0,1,2,3,4,5,6,7,8,9,...,133,134,135,136,137,138,139,140,141,142
0,-3.062435,2.657993,-2.556956,0.262692,1.349017,2.207396,-0.849361,0.223139,0.242783,-0.669382,...,-0.018142,0.036253,0.057949,-0.012082,0.074293,-0.095255,0.083441,0.005482,0.237343,0.141757
1,-2.959758,2.548071,-3.785073,-0.000655,0.823500,2.077908,-0.718883,-0.283912,-0.450002,-1.236451,...,-0.148761,-0.090056,-0.057934,0.156247,0.111973,0.151499,0.303699,-0.114877,0.169689,-0.378170
2,-3.119257,2.421025,-2.617167,-0.159477,1.039031,1.922482,-0.709325,0.520871,0.184966,-0.503594,...,0.093779,-0.071209,0.001362,-0.036972,0.000038,-0.105622,-0.155273,-0.053077,0.110324,0.190837
3,2.256025,5.570979,-3.025784,0.755978,-0.188211,0.414380,0.495592,-1.783867,-1.391488,-0.686283,...,0.377672,0.165077,0.448965,0.192430,-0.135077,-1.300647,1.109791,-0.522040,-0.550669,1.149291
4,0.714818,3.420565,3.955201,-0.200466,1.096371,-0.564256,-0.624619,0.191528,0.560434,-0.028703,...,-0.001318,-0.035898,0.021144,0.027821,0.033642,0.032434,0.032533,0.063395,-0.001620,-0.003008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257116,0.988944,-1.875971,-2.209274,0.246926,0.861873,0.608625,0.421180,-0.376476,0.189801,-0.442871,...,0.010698,-0.024786,0.018701,0.064414,0.151321,-0.007017,-0.032042,-0.005727,-0.001962,0.001368
257117,9.143513,8.235104,0.292868,0.326641,1.088608,1.933650,-1.597647,1.486813,1.344689,0.860239,...,-0.144760,0.034442,0.111121,0.044164,-0.107688,-0.044878,-0.034259,-0.133390,-0.336016,0.081180
257118,5.405474,10.845829,-11.488483,3.416707,0.356870,7.459747,-0.937534,0.316334,0.064270,-0.444535,...,0.214760,-0.396116,-1.340524,-0.018469,-0.040614,0.501742,0.458521,-0.641903,-1.060151,-1.432146
257119,-1.400579,0.946393,1.698620,-1.562481,0.444259,1.022762,0.789702,0.465403,1.066712,0.935065,...,-0.030002,0.033354,0.026005,-0.171941,-0.013256,0.006214,-0.026417,-0.095495,0.089007,-0.012854


In [70]:
pca_df_train = pd.concat([pca_df_train, sensors_train, y_train], axis=1)

In [71]:
from sklearn.preprocessing import OrdinalEncoder

In [72]:
ord_enc = OrdinalEncoder()
pca_df_train["code_sensor"] = ord_enc.fit_transform(pca_df_train[["Sensor"]])
pca_df_train.drop("Sensor", axis=1, inplace=True)

In [73]:
pca_df_train

,0,1,2,3,4,5,6,7,8,9,...,135,136,137,138,139,140,141,142,Day,code_sensor
0,-3.062435,2.657993,-2.556956,0.262692,1.349017,2.207396,-0.849361,0.223139,0.242783,-0.669382,...,0.057949,-0.012082,0.074293,-0.095255,0.083441,0.005482,0.237343,0.141757,3,0.0
1,-2.959758,2.548071,-3.785073,-0.000655,0.823500,2.077908,-0.718883,-0.283912,-0.450002,-1.236451,...,-0.057934,0.156247,0.111973,0.151499,0.303699,-0.114877,0.169689,-0.378170,4,0.0
2,-3.119257,2.421025,-2.617167,-0.159477,1.039031,1.922482,-0.709325,0.520871,0.184966,-0.503594,...,0.001362,-0.036972,0.000038,-0.105622,-0.155273,-0.053077,0.110324,0.190837,3,0.0
3,2.256025,5.570979,-3.025784,0.755978,-0.188211,0.414380,0.495592,-1.783867,-1.391488,-0.686283,...,0.448965,0.192430,-0.135077,-1.300647,1.109791,-0.522040,-0.550669,1.149291,5,0.0
4,0.714818,3.420565,3.955201,-0.200466,1.096371,-0.564256,-0.624619,0.191528,0.560434,-0.028703,...,0.021144,0.027821,0.033642,0.032434,0.032533,0.063395,-0.001620,-0.003008,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257116,0.988944,-1.875971,-2.209274,0.246926,0.861873,0.608625,0.421180,-0.376476,0.189801,-0.442871,...,0.018701,0.064414,0.151321,-0.007017,-0.032042,-0.005727,-0.001962,0.001368,2,959.0
257117,9.143513,8.235104,0.292868,0.326641,1.088608,1.933650,-1.597647,1.486813,1.344689,0.860239,...,0.111121,0.044164,-0.107688,-0.044878,-0.034259,-0.133390,-0.336016,0.081180,5,959.0
257118,5.405474,10.845829,-11.488483,3.416707,0.356870,7.459747,-0.937534,0.316334,0.064270,-0.444535,...,-1.340524,-0.018469,-0.040614,0.501742,0.458521,-0.641903,-1.060151,-1.432146,3,959.0
257119,-1.400579,0.946393,1.698620,-1.562481,0.444259,1.022762,0.789702,0.465403,1.066712,0.935065,...,0.026005,-0.171941,-0.013256,0.006214,-0.026417,-0.095495,0.089007,-0.012854,7,959.0


In [74]:
test = pd.read_csv("data/df_cleared_test.csv")

In [75]:
X_test = test.drop(columns=["Unnamed: 0", "Day"])
y_test = test["Day"]

fets_test = X_test.drop("Sensor", axis=1).columns
sensors_test = X_test.Sensor
X_test = test.loc[:, fets_test].values

X_test = StandardScaler().fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns=fets_test)

In [76]:
#pca = PCA(n_components="mle", svd_solver = 'full')
pcas_test = pca.fit_transform(X_test)
pca_df_test = pd.DataFrame(data=pcas_test)
pca_df_test

,0,1,2,3,4,5,6,7,8,9,...,133,134,135,136,137,138,139,140,141,142
0,1.437692,-1.947014,4.728712,0.622721,-1.303665,0.947684,-0.587125,0.117830,0.090733,0.275980,...,0.007394,0.072311,-0.056124,-0.054061,0.189534,0.164966,0.026812,-0.150058,0.099668,-0.242992
1,-2.937315,-4.113356,1.294441,0.348600,-1.151246,2.247106,-0.641635,-0.428238,-0.919906,0.188466,...,0.047788,-0.017239,0.219050,-0.246551,-0.249995,-0.169026,0.193613,-0.102754,0.411920,-0.210689
2,-2.790194,1.585091,0.845264,-1.775457,-0.102517,0.400486,-0.080375,0.910058,0.562574,-0.510081,...,0.000047,0.003142,0.086537,-0.041396,0.068083,0.025084,-0.036028,0.075458,-0.008786,-0.066942
3,1.451360,-3.441318,4.835274,0.977426,-0.565877,1.671365,-0.582821,-0.222574,-1.076701,0.724774,...,-0.063159,0.000787,-0.000693,-0.076673,-0.053421,-0.019144,0.321653,-0.510008,-0.157452,-0.308516
4,1.470210,-3.227005,5.072361,0.915745,-0.862020,1.702370,-0.377423,-0.045470,-0.685217,1.086605,...,-0.081793,0.041115,-0.543343,0.139882,0.142156,-0.263208,-0.292338,-0.009754,-0.394692,0.615057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166594,-3.055431,3.371925,2.302582,-3.081939,2.639484,-0.518862,-2.997799,1.438312,-0.996456,-0.027406,...,-0.076304,-0.170243,0.103921,0.042943,0.034443,-0.029992,0.027540,-0.041591,-0.057472,0.114948
166595,5.688810,-15.193720,6.006377,3.611802,-0.128681,7.366377,-0.975179,-0.228489,-1.434123,0.725590,...,0.131744,0.051225,-0.072855,0.387669,-0.639141,0.508186,0.910691,-1.149111,-1.011214,1.150463
166596,2.185361,-1.386125,-2.721199,0.447259,-0.027310,0.143807,0.285525,-0.292869,-0.141667,0.450175,...,-0.224268,-0.078656,-0.065013,-0.004757,-0.021434,0.011850,0.043651,-0.013021,0.016750,-0.014106
166597,2.920097,-3.657836,9.106467,2.476527,-0.197666,2.407538,-0.499012,0.149601,0.554357,0.099667,...,0.007223,0.040927,-0.090260,0.048347,-0.005408,0.002383,-0.006204,-0.063608,0.102673,-0.098078


In [77]:
pca_df_test = pd.concat([pca_df_test, sensors_test, y_test], axis=1)

In [79]:
ord_enc = OrdinalEncoder()
pca_df_test["code_sensor"] = ord_enc.fit_transform(pca_df_test[["Sensor"]])
pca_df_test.drop("Sensor", axis=1, inplace=True)

In [80]:
pca_df_test

,0,1,2,3,4,5,6,7,8,9,...,135,136,137,138,139,140,141,142,Day,code_sensor
0,1.437692,-1.947014,4.728712,0.622721,-1.303665,0.947684,-0.587125,0.117830,0.090733,0.275980,...,-0.056124,-0.054061,0.189534,0.164966,0.026812,-0.150058,0.099668,-0.242992,4,0.0
1,-2.937315,-4.113356,1.294441,0.348600,-1.151246,2.247106,-0.641635,-0.428238,-0.919906,0.188466,...,0.219050,-0.246551,-0.249995,-0.169026,0.193613,-0.102754,0.411920,-0.210689,2,0.0
2,-2.790194,1.585091,0.845264,-1.775457,-0.102517,0.400486,-0.080375,0.910058,0.562574,-0.510081,...,0.086537,-0.041396,0.068083,0.025084,-0.036028,0.075458,-0.008786,-0.066942,7,0.0
3,1.451360,-3.441318,4.835274,0.977426,-0.565877,1.671365,-0.582821,-0.222574,-1.076701,0.724774,...,-0.000693,-0.076673,-0.053421,-0.019144,0.321653,-0.510008,-0.157452,-0.308516,4,0.0
4,1.470210,-3.227005,5.072361,0.915745,-0.862020,1.702370,-0.377423,-0.045470,-0.685217,1.086605,...,-0.543343,0.139882,0.142156,-0.263208,-0.292338,-0.009754,-0.394692,0.615057,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166594,-3.055431,3.371925,2.302582,-3.081939,2.639484,-0.518862,-2.997799,1.438312,-0.996456,-0.027406,...,0.103921,0.042943,0.034443,-0.029992,0.027540,-0.041591,-0.057472,0.114948,1,959.0
166595,5.688810,-15.193720,6.006377,3.611802,-0.128681,7.366377,-0.975179,-0.228489,-1.434123,0.725590,...,-0.072855,0.387669,-0.639141,0.508186,0.910691,-1.149111,-1.011214,1.150463,3,959.0
166596,2.185361,-1.386125,-2.721199,0.447259,-0.027310,0.143807,0.285525,-0.292869,-0.141667,0.450175,...,-0.065013,-0.004757,-0.021434,0.011850,0.043651,-0.013021,0.016750,-0.014106,3,959.0
166597,2.920097,-3.657836,9.106467,2.476527,-0.197666,2.407538,-0.499012,0.149601,0.554357,0.099667,...,-0.090260,0.048347,-0.005408,0.002383,-0.006204,-0.063608,0.102673,-0.098078,6,959.0


In [61]:
rf = RandomForestClassifier(random_state=1)

In [83]:
rf.fit(X=pca_df_train.drop("Day", axis=1), y=y_train)

c:\Users\nikla\anaconda3\envs\NLP\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier(random_state=1)

In [85]:
rf.score(X=pca_df_test.drop("Day", axis=1), y=y_test)

c:\Users\nikla\anaconda3\envs\NLP\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.2469282528706655